In [1]:
!sudo pip install pandas_profiling

In [2]:
!sudo pip install gspread

In [3]:
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport

import numpy as np
import string
from google.oauth2 import service_account
import gspread

from pathlib import Path
import os 

import time, datetime, os
import shutil 
import requests

In [4]:
# Check today's date and create a directory file structure for today's date.
today = datetime.date.today()  
todaystr = today.isoformat()   

def create_date_dir():
    i = 0
    d = todaystr
    if not os.path.exists(d):
        os.mkdir(d)
    else:
        '''#if os.path.exists(d):
        #    i += 1
        #    d = d[:10] + '_' + str(i).zfill(2)
        #os.mkdir(d)
        #else:
        #    m = re.findall(r'\w\w$', d)
        #    i = float(m[0:]) + 1
        #    d = d[:10] + '_' + str(i).zfill(2)
        #os.mkdir(d)'''
        shutil.rmtree(d)
        #os.rmdir(d)
        os.mkdir(d)
        
create_date_dir()

# This will point to where the job is being run from NOT the directory where this file lives (mnt/notebooks/scratch), when run as a Domino Job 
current_dir = os.getcwd()
    
## Read in service account credentials - link to whichever json file holds your credentials
gc = gspread.service_account(filename='/mnt/data/odt-data-profiler-069defc7abf3.json')

## Spreadsheet key of your destination 
sh2 = gc.open_by_key('1gtMHAQkfufSOPb3hpThM_aZ0rwV8NG-CeVh3BY2dorQ')

# Read data from worksheet
worksheet2 = sh2.get_worksheet(0)
list_of_lists = worksheet2.get_all_records()



In [5]:
list_of_lists

[{'agency': 'DOB', 'dataset': 'Certificate of Occupancy', '4x4': 'pkdm-hqz6'}]

In [10]:
def profile_data(list_of_lists):
    for i in range (0,len(list_of_lists)):
        agency_name = list_of_lists[i]['agency']
        dataset_name = list_of_lists[i]['dataset']
        url = list_of_lists[i]['api_endpoint']
        dataset_url = f'https://data.cityofnewyork.us/resource/{url}.json'
        #count = list_of_lists[i]['count']
        #limit = dataset_url+"?$select=count(*)
        chunk_size = 1000
        offset = 0
        reached_end_of_dataset = False
        tmp_dfs = []
        while not reached_end_of_dataset:
            chunk_url = f'{dataset_url}?$limit={chunk_size}&$offset={offset}'
            tmp_r = requests.get(chunk_url, auth=("8febnpgtxedbep4no9oqegrce","36g68l45jhsdzeu08j3i44j4nxn2t65ua1gw4o0xwa9o17ownv"), timeout=None) #pd.read_json(chunk_url)
            tmp = pd.DataFrame(tmp_r.json())
            tmp_dfs.append(tmp)
            #print(chunk_url)
    
            if tmp.shape[0] == chunk_size:
                if offset <= 9860000:
                    offset += chunk_size
                else:
                    break
    
            else: # if got the last chunk, should be smaller than the chunk size 
                reached_end_of_dataset = True

        out_df = pd.concat(tmp_dfs, axis=0)
        #r = requests.get(dataset_url+f"?$limit=500000", auth=("8febnpgtxedbep4no9oqegrce","36g68l45jhsdzeu08j3i44j4nxn2t65ua1gw4o0xwa9o17ownv"), timeout=None)
        prl = ProfileReport(out_df, title=f"{dataset_name}_Data_Profiler", html={"style": {"full_width": True}}, sort=None,
            correlations=None,
            interactions=None,
            missing_diagrams={
                "bar": False,
                "matrix": True,
                "heatmap": False,
                "dendrogram": False,
            })
        out = prl.to_file(current_dir + "/" + todaystr + "/" + f"{todaystr}_{agency_name}_{dataset_name}.html")


In [12]:
### TESTING

# Function to profile each dataset
def profile_data(list_of_lists):
    for i in range (0,len(list_of_lists)):
        agency_name = list_of_lists[i]['agency'] # agency field for each row - will be used in the report name
        dataset_name = list_of_lists[i]['dataset'] # dataset name for each row - will be used in the report name
        url = list_of_lists[i]['4x4'] # 4x4 for each row - will be used to identify the dataset and read in the data from the API endpoint
        dataset_url = f'https://data.cityofnewyork.us/resource/{url}.json' # defining the url using the 4x4 for each row
        chunk_size = 100000 # the chunk size is the limit that will be set for the get requests. Only 100000 rows are read at a given time. 
        offset = 0
        reached_end_of_dataset = False
        tmp_dfs = []
        while not reached_end_of_dataset:
            chunk_url = f'{dataset_url}?$limit={chunk_size}&$offset={offset}'
            tmp_r = requests.get(chunk_url, auth=("8febnpgtxedbep4no9oqegrce","36g68l45jhsdzeu08j3i44j4nxn2t65ua1gw4o0xwa9o17ownv"), timeout=None)
            tmp = pd.DataFrame(tmp_r.json())
            tmp_dfs.append(tmp)
            print(chunk_url)
            
            if (tmp.shape[0] == chunk_size): 
                if (offset < 10000000): # Offset and limit are used together to incrementally read 100000 rows at a time if we have not reached the end of the dataset. 
                    offset += chunk_size
                else:
                    break
        
            else: # if got the last chunk, should be smaller than the chunk size 
                reached_end_of_dataset = True

        out_df = pd.concat(tmp_dfs, axis=0) # chunks of dataset are concatenated to get the full dataset. NOTE THAT THIS STEP FAILS FOR VERY LARGE DATASETS LARGER THAN ~ 10 MILLION ROWS.
        prl = ProfileReport(out_df, minimal=True, show_variable_description=False, variables={ "descriptions": {} } , title=f"{dataset_name}_Profile", sort=None,
            correlations=None,
            interactions=None,
            missing_diagrams=None)
        
        out = prl.to_file(current_dir + "/" + todaystr + "/" + f"{todaystr}_{agency_name}_{dataset_name}.html") # output the report to the appropriate directory.

In [13]:
profile_data(list_of_lists)

https://data.cityofnewyork.us/resource/pkdm-hqz6.json?$limit=100000&$offset=0


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
!which python

/opt/conda/bin/python


In [16]:
import sys
sys.executable

'/opt/conda/bin/python'

In [20]:
url = 'https://data.cityofnewyork.us/resource/j34j-vqvt.json'
    
chunk_size = 1000
offset = 0
reached_end_of_dataset = False


tmp_dfs = []

while not reached_end_of_dataset:
    chunk_url = f'{url}?$limit={chunk_size}&$offset={offset}'
    
    tmp = pd.read_json(chunk_url)
    tmp_dfs.append(tmp)
    
    if tmp.shape[0] == chunk_size:
        offset += chunk_size
        
    else: # if got the last chunk, should be smaller than the chunk size 
        reached_end_of_dataset = True

out_df = pd.concat(tmp_dfs, axis=0)

In [21]:
out_df

,yearmonth,incidentclassification,incidentborough,incidentcount,averageresponsetime
0,2009/07,All Fire/Emergency Incidents,Citywide,40850,04:27
1,2009/07,All Fire/Emergency Incidents,Manhattan,10709,04:32
2,2009/07,All Fire/Emergency Incidents,Bronx,8137,04:37
3,2009/07,All Fire/Emergency Incidents,Staten Island,2205,04:45
4,2009/07,All Fire/Emergency Incidents,Brooklyn,11505,04:01
...,...,...,...,...,...
363,FY 2017,Structural Fires,Manhattan,5669,04:26
364,FY 2017,Structural Fires,Bronx,5829,04:22
365,FY 2017,Structural Fires,Staten Island,1266,04:28
366,FY 2017,Structural Fires,Brooklyn,7949,03:42


In [ ]:
import random

filename = "data.txt"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 10000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pandas.read_csv(filename, skiprows=skip)